# Cuaderno: Extracción de Conocimiento con Grafos usando LLMs

**Objetivo:** Transformar el texto no estructurado de las noticias en un grafo de conocimiento estructurado. Usaremos un Modelo de Lenguaje Grande (LLM) disponible en nuestro endpoint, **`databricks-gemma-3-12b`**, para identificar automáticamente las entidades y sus relaciones.

**Fases del Taller:**
1.  **Diseñar un Prompt:** Crear la instrucción perfecta para que el LLM actúe como un extractor de conocimiento.
2.  **Aplicar el Modelo a Escala:** Usar la función `ai_query` para ejecutar el LLM sobre nuestra tabla `noticias_silver`.
3.  **Modelar el Grafo:** Parsear la salida del LLM y almacenarla en tablas Delta de nodos (`entidades_gold`) y aristas (`relaciones_gold`).
4.  **Analizar el Grafo:** Utilizar PySpark puro para realizar consultas sobre nuestra nueva red de conocimiento.

### Paso 1: El Arte del Prompt Engineering

La clave para que un LLM realice una tarea específica es darle una **instrucción clara y precisa (un "prompt")**.

Le pediremos al modelo `databricks-gemma-3-12b` que lea un artículo y nos devuelva una estructura que podamos procesar fácilmente: un **formato JSON**.

Nuestro prompt le pedirá al modelo dos cosas:
1.  Una lista de `entidades` con su nombre y tipo (PERSONA, ORGANIZACION, LUGAR).
2.  Una lista de `relaciones` en formato de triplete: `[entidad_origen, tipo_de_relacion, entidad_destino]`.

In [0]:
%sql
-- Configuración Inicial

-- Nos aseguramos de estar en la base de datos correcta.
USE curso_arquitecturas;

-- Damos un vistazo rápido a la tabla silver que creamos en la sección anterior.
-- La columna 'contenido' será la entrada para nuestro LLM.
SELECT titulo, contenido FROM noticias_silver LIMIT 10;

### Paso 2: Usando `ai_query` para Interactuar con el Modelo Gemma

Ahora que has confirmado que el endpoint `databricks-gemma-3-12b` está disponible, podemos crear una función SQL que lo invoque con nuestro prompt. Esto abstrae la complejidad y hace que nuestro código principal sea más legible.

In [0]:
%sql
-- Creación de una Función SQL para Extraer el Grafo con Gemma

-- Esta función toma el texto de una noticia como entrada y le pide al LLM
-- que devuelva las entidades y relaciones en formato JSON.

CREATE OR REPLACE FUNCTION EXTRACT_GRAPH_JSON(content STRING)
RETURNS STRING
RETURN AI_QUERY(
  'databricks-gemma-3-12b', 
  CONCAT(
    'Eres un experto analista de inteligencia que extrae conocimiento de textos. ',
    'Analiza la siguiente noticia y extrae las principales entidades y sus relaciones. ',
    'Responde únicamente con un objeto JSON que contenga dos claves: "entidades" y "relaciones". ',
    'Para las entidades, usa los tipos: PERSONA, ORGANIZACION, LUGAR. ',
    'Para las relaciones, usa el formato [sujeto, predicado, objeto]. ',
    'Noticia: ', content
  )
);

In [0]:
%sql
-- Creación de una Función SQL para Extraer el Grafo con Gemma

-- Esta función toma el texto de una noticia como entrada y le pide al LLM
-- que devuelva las entidades y relaciones en formato JSON. databricks-gpt-oss-120b

CREATE OR REPLACE FUNCTION EXTRACT_GRAPH_JSON_GPT(content STRING)
RETURNS STRING
RETURN AI_QUERY(
  'databricks-gpt-oss-120b', 
  CONCAT(
    'Eres un experto analista de inteligencia que extrae conocimiento de textos. ',
    'Analiza la siguiente noticia y extrae las principales entidades y sus relaciones. ',
    'Responde únicamente con un objeto JSON que contenga dos claves: "entidades" y "relaciones". ',
    'Para las entidades, usa los tipos: PERSONA, ORGANIZACION, LUGAR. ',
    'Para las relaciones, usa el formato [sujeto, predicado, objeto]. ',
    'Noticia: ', content
  )
);

In [0]:
%sql
SELECT 
EXTRACT_GRAPH_JSON("En el bosque de la china Xi JINPING se perdio y como GUSTAVO estaba perdido se encontraron los dos")
AS grafo_extraido;

In [0]:
%sql
SELECT 
EXTRACT_GRAPH_JSON_GPT("En el bosque de la china Xi JINPING se perdio y como GUSTAVO estaba perdio se encontraron los dos")
AS grafo_extraido;

### Paso 3: Aplicando el LLM sobre un Conjunto de Noticias

Ahora que tenemos nuestra función `EXTRACT_GRAPH_JSON`, podemos aplicarla a la columna `contenido`. Para este taller, limitaremos el proceso a una pequeña muestra de 5 noticias para demostrar el concepto de forma rápida.

In [0]:
%sql
-- Ejecutamos el LLM sobre 5 noticias y guardamos la salida JSON en una tabla temporal

CREATE OR REPLACE TEMP VIEW noticias_con_grafo_json AS
SELECT
  id_noticia,
  titulo,
  EXTRACT_GRAPH_JSON(contenido) AS grafo_json
FROM curso_arquitecturas.noticias_silver
LIMIT 50;

-- Revisemos la salida cruda del LLM
SELECT titulo, grafo_json FROM noticias_con_grafo_json;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW noticias_con_grafo_json AS
SELECT
  id_noticia,
  titulo,
  EXTRACT_GRAPH_JSON_GPT(contenido) AS grafo_json
FROM curso_arquitecturas.noticias_silver
LIMIT 50;

-- Revisemos la salida cruda del LLM
SELECT titulo, grafo_json FROM noticias_con_grafo_json;

### Paso 4: Parseando el JSON para Estructurar el Grafo

La salida del LLM es una cadena de texto en formato JSON. Nuestro siguiente paso es "desempacar" (parsear) este JSON y cargarlo en nuestras tablas finales de la capa Oro:
* `entidades_gold` (para los nodos)
* `relaciones_gold` (para las aristas)

Usaremos las funciones de Spark SQL `from_json` y `explode` para esta tarea.

In [0]:
%sql
-- Creación de las Tablas Finales en la Capa Oro

CREATE OR REPLACE TABLE entidades_gold (
  id_noticia STRING,
  nombre_entidad STRING,
  tipo_entidad STRING
);

CREATE OR REPLACE TABLE relaciones_gold (
  id_noticia STRING,
  entidad_origen STRING,
  relacion STRING,
  entidad_destino STRING
);

In [0]:
%sql
-- Parsear y Cargar las ENTIDADES (Nodos) - VERSIÓN CORREGIDA FINAL

-- Añadimos un paso de limpieza (clean_json) usando regexp_replace para
-- eliminar los ```json y ``` que el LLM añade a la salida.

INSERT INTO entidades_gold
WITH cleaned_json AS (
  SELECT
    id_noticia,
    regexp_replace(grafo_json, '(^```json\\n|\\n```$)', '') AS json_limpio
  FROM noticias_con_grafo_json
  WHERE grafo_json IS NOT NULL
),
parsed_json AS (
  SELECT
    id_noticia,
    from_json(json_limpio, 'STRUCT<entidades: ARRAY<STRUCT<tipo: STRING, nombre: STRING>>, relaciones: ARRAY<ARRAY<STRING>>>') AS data
  FROM cleaned_json
)
SELECT
  id_noticia,
  entidad.nombre AS nombre_entidad,
  entidad.tipo AS tipo_entidad
FROM parsed_json
LATERAL VIEW EXPLODE(data.entidades) AS entidad;

In [0]:
%sql
SELECT * FROM entidades_gold

In [0]:
%sql
-- Parsear y Cargar las RELACIONES (Aristas) - VERSIÓN CORREGIDA Y ROBUSTA

-- Aplicamos la misma lógica de limpieza aquí para asegurar la consistencia
-- y robustez del pipeline.

INSERT INTO relaciones_gold
WITH cleaned_json AS (
  SELECT
    id_noticia,
    regexp_replace(grafo_json, '(^```json\\n|\\n```$)', '') AS json_limpio
  FROM noticias_con_grafo_json
  WHERE grafo_json IS NOT NULL
),
parsed_json AS (
  SELECT
    id_noticia,
    from_json(json_limpio, 'STRUCT<entidades: ARRAY<STRUCT<tipo: STRING, nombre: STRING>>, relaciones: ARRAY<ARRAY<STRING>>>') AS data
  FROM cleaned_json
)
SELECT
  id_noticia,
  get(relacion, 0) AS entidad_origen, -- Acceso seguro al primer elemento
  get(relacion, 1) AS relacion,       -- Acceso seguro al segundo elemento
  get(relacion, 2) AS entidad_destino -- Acceso seguro al tercer elemento (devolverá NULL si no existe)
FROM parsed_json
LATERAL VIEW EXPLODE(data.relaciones) AS relacion;

In [0]:
%sql
-- Verificación del Grafo Estructurado: Aristas

SELECT * FROM relaciones_gold;

### Paso 5: Análisis de la Red con PySpark Puro

Ahora que el grafo está modelado en tablas Delta, usaremos PySpark para analizarlo. Calcularemos los "grados" de los nodos para identificar las entidades más importantes o centrales en nuestra pequeña red de noticias.

In [0]:
# Análisis 1: Calculando los Grados de los Nodos con PySpark

from pyspark.sql.functions import col, desc

# Cargamos la tabla de aristas
aristas_df = spark.table("relaciones_gold")

In [0]:
# Para calcular el grado, contamos cuántas veces aparece una entidad
# tanto en la columna de origen como en la de destino.
endpoints_df = aristas_df.select(col("entidad_origen").alias("entidad")) \
    .unionAll(aristas_df.select(col("entidad_destino").alias("entidad")))

endpoints_df.show()

In [0]:
# Agrupamos por entidad y contamos las ocurrencias.
grados_df = endpoints_df.groupBy("entidad").count() \
    .withColumnRenamed("count", "degree")

grados_df.orderBy(desc("degree")).show()


In [0]:

# Mostramos las entidades más conectadas.
print("Entidades más centrales (con más conexiones) calculadas con PySpark:")
display(grados_df.orderBy(desc("degree")))

### Paso 6: Visualización con `networkx`

Finalmente, aplicamos el patrón híbrido: usamos PySpark para el cálculo distribuido y ahora traemos el resultado (que ya es pequeño) al driver para visualizarlo con `networkx`.

In [0]:
# Traemos el subconjunto de datos a pandas para usar con networkx

import networkx as nx
import matplotlib.pyplot as plt

# Recolectamos los datos (ya son pequeños) al driver
nodos_pd = spark.table("curso_arquitecturas.entidades_gold").select("nombre_entidad").distinct().toPandas()
aristas_pd = spark.table("curso_arquitecturas.relaciones_gold").toPandas()

nodos_pd=nodos_pd.iloc[:20]
aristas_pd=aristas_pd.iloc[:100]
# Creamos el grafo con networkx
G = nx.from_pandas_edgelist(aristas_pd.dropna(subset=["entidad_origen", "entidad_destino"]), "entidad_origen", "entidad_destino")

# Visualizamos el grafo
plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G, k=0.8, iterations=50)
nx.draw(G, pos, with_labels=True, node_size=1000, node_color='skyblue', font_size=9, font_weight='bold', width=1.5, edge_color='gray')
plt.title("Red de Conocimiento Extraída de las Noticias", size=20)
plt.show()

### Conclusión de la Sección

¡Felicidades! Has completado un flujo de trabajo de IA de vanguardia:

1.  **Identificaste** un recurso de cómputo de IA disponible (`databricks-gemma-3-12b`).
2.  **Diseñaste un prompt** para instruir al modelo a realizar una tarea de extracción estructurada.
3.  **Ejecutaste el LLM** a través de SQL para convertir texto no estructurado en JSON.
4.  **Parseaste y modelaste** la salida en un grafo con nodos y aristas en la capa Oro.
5.  **Analizaste y visualizaste** la red resultante.

Este proceso es la base de sistemas de inteligencia, análisis de riesgo y descubrimiento de información, demostrando cómo una arquitectura Lakehouse moderna puede ir mucho más allá del BI tradicional.